# Extraction de fichiers d'un type MIME donné à partir d'un W/ARC
Le script suivant prend en entrée un dossier où se trouvent des fichiers ARC ou WARC gzippés. Il cherche dans chacun des fichiers W/ARC (ARC ou WARC) des enregistrements ayant un type MIME particulier. Il les extrait à l'emplacement du script, en les renommant à partir du nom du WARC et en ajoutant un numéro séquentiel.

La première opération est d'installer le package [warcio](https://github.com/webrecorder/warcio).

On importe ensuite les modules de base de python dont on a besoin : [os.path](https://docs.python.org/fr/3/library/os.path.html#module-os.path) pour gérer les chemins et noms de fichiers, [glob](https://docs.python.org/fr/3/library/glob.html) pour l'identification de fichiers selon un motif qui gère le caractère joker "*" et enfin [io](https://docs.python.org/fr/3/library/io.html#module-io) qui permet notamment d'écrire des flux binaires dans des fichiers.

Enfin, on importe la classe archiveiterator du module warcio pour parser des fichiers W/ARC.

In [ ]:
from os import path
import glob
import io

from warcio import archiveiterator

: 

La première fonction (exploratoire) prend en paramètres le type MIME recherché et le fichier W/ARC cible. Elle ouvre le fichier W/ARC en lecture et en mode binaire et parcourt tous les enregistrements. Elle ne considère que les enregistrements qui sont de type "response" (réponse du serveur) et dont l'en-tête HTTP(S) contient un attribut `Content-Type` qui correspond au type MIME recherché. Dans ce cas, elle affiche l'URL où l'enregistrement a été collecté.

Dans le cas d'une erreur liée à l'absence de l'attribut `Content-Type`, elle se contente de passer à l'enregistrement suivant.

In [ ]:
def print_warc_records_uri(mimetype, warc_file):
    """This function searches for all records of a given mimetype in the WARC file
and extracts them."""
    with open(warc_file, 'rb') as records_file:
        for record in archiveiterator.ArchiveIterator(records_file, arc2warc=True):
            try:
                if record.rec_type == 'response' and record.http_headers.get_header('Content-Type') ==  mimetype:
                    print(record.rec_headers.get_header('WARC-Target-URI'))
            except AttributeError:
                pass

On appelle ensuite la fonction avec une boucle parcourant tous les fichiers d'un dossier donné, en spécifiant les fichiers qu'on souhaite traiter avec un motif de type "*arc.gz" qui trouvera tous les fichiers ARC ou WARC gzippés (s'ils sont bien nommés, évidemment).

On notera l'utilisation de l'expression `os.path.expanduser('~') + 'chemin/vers/le/fichier')` pour accéder aux fichiers situés dans l'espace du groupe Formats.

In [ ]:
for warc in glob.glob('*arc'):
    print_warc_records_uri('text/html', warc)

On déclare une seconde fonction, sur le principe assez similaire, qui cette fois-ci extraiera les enregistrements sous forme de fichiers. La fonction déclare un compteur `i` permettant de renommer séquentiellement les fichiers. Elle capture dans une variable `stream` le flux de données de chaque enregistrement (ce que l'on nomme le _payload_ de l'enregistrement) et le déclare comme flux de données binaires. Elle ouvre ensuite un fichier et y inscrit le flux en question.

In [ ]:
def extract_warc_records(mimetype, warc_file):
    """This function searches for all records of a given mimetype in the WARC file
and extracts them."""
    with open(warc_file, 'rb') as records_file:
        i = 0
        for record in archiveiterator.ArchiveIterator(records_file, arc2warc=True):
            try:
                if record.rec_type == 'response' and record.http_headers.get_header('Content-Type') ==  mimetype:
                    stream = io.BytesIO(record.content_stream().read())
                    with open(path.basename(warc_file) + "_" + str(i) + ".apk", "wb") as f:
                        f.write(stream.getbuffer())
                    i += 1
            except AttributeError:
                pass

De la même façon que pour la fonction précédente, on l'appelle par son petit nom.

In [ ]:
for warc in glob.glob('*arc'):
    extract_warc_records('text/html', warc)

Et le tour est joué !

Si vous voulez jouer avec des exemples plus conséquents et réels, Internet Archive met à disposition des WARC exemples plus conséquents à explorer à [cette adresse](https://archive.org/download/testWARCfiles).